In [66]:
import pandas as pd
import sqlite3
from collections import defaultdict as dd
import json
import dbga
print(dbga.__version__)

1.0.4


In [67]:
conn = sqlite3.connect('../databases/store.db')
curs = conn.cursor()

In [68]:
my_sql = dd(dict)

In [69]:
pd.read_sql("PRAGMA table_info(tCust);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,cust_id,INTEGER,0,None,1
1,1,first,TEXT,1,None,0
2,2,last,TEXT,1,None,0
3,3,addr,TEXT,1,None,0
4,4,zip,TEXT,1,None,0


In [70]:
pd.read_sql("PRAGMA table_info(tZip);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,zip,TEXT,0,None,1
1,1,city,TEXT,1,None,0
2,2,state_id,TEXT,1,None,0


In [71]:
pd.read_sql("PRAGMA table_info(tState);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,state_id,TEXT,0,None,1
1,1,state,TEXT,1,None,0


In [72]:
pd.read_sql("PRAGMA table_info(tOrder);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,1
1,1,cust_id,INTEGER,1,None,0
2,2,date,TEXT,1,None,0


In [73]:
pd.read_sql("PRAGMA table_info(tOrderDetail);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,1
1,1,prod_id,INTEGER,0,None,2
2,2,qty,INTEGER,1,None,0


In [74]:
pd.read_sql("PRAGMA table_info(tProd);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,prod_id,INTEGER,0,None,1
1,1,prod_desc,TEXT,1,None,0
2,2,unit_price,NUMERIC,1,None,0


---

1) For each customer, find their most recent order (including all products on that order) and return:

- cust_id
- first
- last
- order_id
- date
- prod_id
- prod_desc
- qty
- unit_price
- total = qty*unit_price

Order the results by cust_id, order_id and prod_id, all ascending.

In [75]:
sql1 = """
SELECT cust_id, first, last, order_id, date, prod_id, prod_desc, qty, unit_price, (qty * unit_price) AS total
FROM tCust
JOIN (SELECT * FROM tOrder WHERE (cust_id, date) IN (SELECT cust_id, MAX(date) FROM tOrder GROUP BY cust_id)) USING(cust_id)
JOIN tOrderDetail USING(order_id)
JOIN tProd USING(prod_id)
ORDER BY cust_id, order_id, prod_id ASC
;"""


pd.read_sql(sql1, conn)

,cust_id,first,last,order_id,date,prod_id,prod_desc,qty,unit_price,total
0,1,Bib Fortuna,Walker,150,2019-02-18,302,Nut,4,0.25,1.00
1,1,Bib Fortuna,Walker,150,2019-02-18,307,Sandpaper,8,3.00,24.00
2,1,Bib Fortuna,Walker,150,2019-02-18,314,Mallet,8,12.00,96.00
3,2,Unkar Plutt,Jennings,140,2019-02-15,304,Bolt,4,1.00,4.00
4,2,Unkar Plutt,Jennings,140,2019-02-15,310,Scraper,6,7.99,47.94
...,...,...,...,...,...,...,...,...,...,...
473,134,Tey How,Mcdaniel,168,2019-02-28,318,Hacksaw,6,19.99,119.94
474,135,Jobal,Johnston,170,2019-02-28,300,Washer,8,0.10,0.80
475,135,Jobal,Johnston,170,2019-02-28,304,Bolt,6,1.00,6.00
476,135,Jobal,Johnston,170,2019-02-28,316,Anvil,6,16.20,97.20


In [76]:
my_sql['q1']['sql'] = sql1

---

2) For each customer, find all products they have never purchased with a unit_price > 5.

Return:

- cust_id
- prod_id
- unit_price

Order the results by cust_id and prod_id, both ascending.

In [77]:
sql2 = """
SELECT cust_id, prod_id, unit_price
FROM tCust
JOIN tProd 
WHERE unit_price > 5 AND NOT EXISTS 
(
    SELECT 1
    FROM tOrder
    JOIN tOrderDetail USING(order_id)
    WHERE tOrder.cust_id = tCust.cust_id AND tOrderDetail.prod_id = tProd.prod_id
)
ORDER BY cust_id, prod_id ASC
;"""

pd.read_sql(sql2, conn)

,cust_id,prod_id,unit_price
0,1,310,7.99
1,1,311,8.27
2,1,312,10.99
3,1,313,11.00
4,1,315,15.99
...,...,...,...
2318,135,325,50.00
2319,135,326,69.00
2320,135,327,80.00
2321,135,328,300.00


In [78]:
my_sql['q2']['sql'] = sql2

In [79]:
with open('assignment3.data302', 'w') as f:
    json.dump(my_sql, f)

In [83]:
dbga.validate_submission_format(path_db = '../databases/store.db',
                                path_submission = 'assignment3.data302',
                                num_questions = 2)

TypeError: format_exception() missing 2 required positional arguments: 'value' and 'tb'